In [3]:
# Cell 1: Import necessary libraries
import pandas as pd
import sqlite3
import os

# Set up display options for pandas DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully.")

# Cell 2: Define file paths
# Assuming CSVs are in the same directory as the notebook
csv_file1 = 'products_day1.csv'
csv_file2 = 'products_day2.csv'

print(f"Input CSV files: {csv_file1}, {csv_file2}")

# Cell 3: Read CSV files into pandas DataFrames
try:
    df_day1 = pd.read_csv(csv_file1)
    df_day2 = pd.read_csv(csv_file2)
    print("\nCSV files loaded into DataFrames:")
    print("\n--- products_day1.csv (df_day1) ---")
    print(df_day1)
    print("\n--- products_day2.csv (df_day2) ---")
    print(df_day2)
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure the CSV files are in the correct directory.")
    exit() # Exit if files are not found

# Cell 4: Create an in-memory SQLite database and load DataFrames as tables
conn = sqlite3.connect(':memory:') # Create an in-memory SQLite database

# Load df_day1 into a SQL table named products_day1_tbl
df_day1.to_sql('products_day1_tbl', conn, if_exists='replace', index=False)
# Load df_day2 into a SQL table named products_day2_tbl
df_day2.to_sql('products_day2_tbl', conn, if_exists='replace', index=False)

print("\nDataFrames successfully loaded into SQLite tables: products_day1_tbl, products_day2_tbl")

# Cell 5: SQL Query to Identify Full Rows Added or Removed
# Identify added rows: product_id present in day2 but NOT in day1
sql_added_rows = """
SELECT
    t2.*,
    'ADDED' AS change_type
FROM
    products_day2_tbl AS t2
WHERE
    t2.product_id NOT IN (SELECT product_id FROM products_day1_tbl)
"""

# Identify removed rows: product_id present in day1 but NOT in day2
sql_removed_rows = """
SELECT
    t1.*,
    'REMOVED' AS change_type
FROM
    products_day1_tbl AS t1
WHERE
    t1.product_id NOT IN (SELECT product_id FROM products_day2_tbl)
"""

# Union both results to get all full row changes
sql_full_row_changes = f"""
{sql_added_rows}
UNION ALL
{sql_removed_rows}
ORDER BY product_id, change_type
"""

print("\n--- Full Row Changes (ADDED/REMOVED) ---")
full_row_changes_df = pd.read_sql_query(sql_full_row_changes, conn)
print(full_row_changes_df)

# Cell 6: SQL Query to Detect Column-Level Changes for Matching Rows
sql_column_changes = """
SELECT
    COALESCE(t1.product_id, t2.product_id) AS product_id,
    CASE
        WHEN t1.name != t2.name THEN 'name'
        WHEN t1.category != t2.category THEN 'category'
        WHEN t1.price != t2.price THEN 'price'
        WHEN t1.stock != t2.stock THEN 'stock'
        ELSE NULL
    END AS column_name,
    CASE
        WHEN t1.name != t2.name THEN t1.name
        WHEN t1.category != t2.category THEN t1.category
        WHEN t1.price != t2.price THEN CAST(t1.price AS TEXT) -- Cast to text for consistency
        WHEN t1.stock != t2.stock THEN CAST(t1.stock AS TEXT) -- Cast to text for consistency
        ELSE NULL
    END AS old_value,
    CASE
        WHEN t1.name != t2.name THEN t2.name
        WHEN t1.category != t2.category THEN t2.category
        WHEN t1.price != t2.price THEN CAST(t2.price AS TEXT)
        WHEN t1.stock != t2.stock THEN CAST(t2.stock AS TEXT)
        ELSE NULL
    END AS new_value
FROM
    products_day1_tbl AS t1
INNER JOIN
    products_day2_tbl AS t2
ON
    t1.product_id = t2.product_id
WHERE
    t1.name != t2.name OR
    t1.category != t2.category OR
    t1.price != t2.price OR
    t1.stock != t2.stock
ORDER BY
    product_id, column_name;
"""

print("\n--- Column-Level Differences ---")
column_level_changes_df = pd.read_sql_query(sql_column_changes, conn)
print(column_level_changes_df)

# Cell 7: Close the database connection
conn.close()
print("\nDatabase connection closed.")

Libraries imported successfully.
Input CSV files: products_day1.csv, products_day2.csv

CSV files loaded into DataFrames:

--- products_day1.csv (df_day1) ---
   product_id               name     category  price  stock
0         101     Wireless Mouse  Electronics  25.99    120
1         102       Water Bottle      Kitchen  10.50    200
2         103  Bluetooth Speaker  Electronics  49.99     75
3         104           Yoga Mat      Fitness  20.00     60
4         105          Desk Lamp    Furniture  35.00     90
5         106           Notebook   Stationery   2.99    300

--- products_day2.csv (df_day2) ---
   product_id               name     category  price  stock
0         101     Wireless Mouse  Electronics  23.99    120
1         102       Water Bottle      Kitchen  10.50    180
2         103  Bluetooth Speaker  Electronics  49.99     75
3         104           Yoga Mat      Fitness  20.00     60
4         105          Desk Lamp    Furniture  37.00     90
5         107        Sma